# AI SpillGuard Oil Spill Detection - Week 3-4 Implementation

## Milestone 3: Week 3-4 Module Implementation

**Module 3: Model Development (Segmentation and Classification)**
- Design and implement deep learning models like U-Net or CNN-based architectures
- Customize input layers to handle single-channel SAR or multi-channel satellite data
- Build the segmentation pipeline to predict oil spill regions from raw image inputs

**Module 4: Training and Evaluation**
- Train the model using training datasets with real-time augmentation and validation
- Implement loss functions like Dice Loss and Binary Cross-Entropy
- Evaluate the model using metrics such as Accuracy, IoU, Dice Coefficient, Precision, and Recall
- Fine-tune hyperparameters based on validation performance

## Module 3: Model Development (Segmentation and Classification)

### Task 3.1: Environment Setup and Imports

In [1]:
# Setup environment and imports
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import random

# Deep learning imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torch.utils.data import random_split

# Set random seeds for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Using device: {device}")

# Set dataset paths
data_root = Path('../data')
for split in ['train', 'val', 'test']:
    for folder in ['images', 'masks']:
        path = data_root / split / folder
        count = len(list(path.glob('*.*')))
        print(f"📂 {split}/{folder}/: {count} files")

🖥️ Using device: cpu
📂 train/images/: 20 files
📂 train/masks/: 20 files
📂 val/images/: 8 files
📂 val/masks/: 8 files
📂 test/images/: 5 files
📂 test/masks/: 5 files


### Task 3.2: Dataset Class Implementation

Create a PyTorch dataset for loading oil spill images and masks:

In [2]:
class OilSpillDataset(Dataset):
    """Oil Spill segmentation dataset"""
    
    def __init__(self, image_dir, mask_dir, transform=None):
        """Initialize the dataset.
        
        Args:
            image_dir: Path to directory containing images
            mask_dir: Path to directory containing masks
            transform: Optional data augmentation transforms
        """
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        
        self.image_paths = sorted(list(self.image_dir.glob('*.jpg')))
        self.mask_paths = []
        
        # Ensure corresponding masks exist for each image
        for img_path in self.image_paths:
            mask_filename = img_path.stem + ".png"  # Change extension to .png
            mask_path = self.mask_dir / mask_filename
            
            if mask_path.exists():
                self.mask_paths.append(mask_path)
            else:
                # No corresponding mask, remove from dataset
                self.image_paths.remove(img_path)
        
        # Verify dataset size
        assert len(self.image_paths) == len(self.mask_paths), "Mismatch in number of images and masks"
        
    def __len__(self):
        """Return the number of samples in the dataset"""
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        """Get a sample from the dataset"""
        # Load image and mask
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
        
        # Load with OpenCV
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        
        # Load mask (binary)
        mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        
        # Ensure binary mask
        mask = (mask > 0).astype(np.float32)
        
        # Apply transforms if specified
        if self.transform is not None:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        else:
            # Default preprocessing
            # Normalize image to [0, 1]
            image = image.astype(np.float32) / 255.0
            
            # Convert to PyTorch tensors
            image = torch.from_numpy(image).permute(2, 0, 1)  # (H,W,C) -> (C,H,W)
            mask = torch.from_numpy(mask).unsqueeze(0)  # (H,W) -> (1,H,W)
        
        return image, mask

# Try to import albumentations for advanced augmentation (optional)
try:
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    
    # Define data augmentation transformations for training
    def get_training_transforms(height=256, width=256):
        return A.Compose([
            A.Resize(height=height, width=width),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=30, p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.GaussianBlur(blur_limit=(3, 7), p=0.3),  # Simulate SAR speckle noise
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
            ToTensorV2(),
        ])
    
    # Define validation/test transformations (no augmentation)
    def get_validation_transforms(height=256, width=256):
        return A.Compose([
            A.Resize(height=height, width=width),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # ImageNet normalization
            ToTensorV2(),
        ])
        
    has_albumentation = True
    print("✅ Using Albumentations for data augmentation")
    
except ImportError:
    print("⚠️ Albumentations not available, using basic transformations")
    has_albumentation = False
    
    # Define PyTorch transformations (simpler but less powerful)
    class BasicTransforms:
        """Basic transformations for data augmentation"""
        def __init__(self, is_train=True):
            self.is_train = is_train
        
        def __call__(self, image, mask):
            # Convert to float32 and normalize
            image = image.astype(np.float32) / 255.0
            mask = mask.astype(np.float32)
            
            if self.is_train:
                # Simple data augmentation for training
                if random.random() > 0.5:
                    image = cv2.flip(image, 1)  # Horizontal flip
                    mask = cv2.flip(mask, 1)
                    
                if random.random() > 0.5:
                    image = cv2.flip(image, 0)  # Vertical flip
                    mask = cv2.flip(mask, 0)
            
            # Convert to PyTorch tensors
            image = torch.from_numpy(image).permute(2, 0, 1)  # (H,W,C) -> (C,H,W)
            mask = torch.from_numpy(mask).unsqueeze(0)  # (H,W) -> (1,H,W)
            
            return {'image': image, 'mask': mask}
    
    def get_training_transforms():
        return BasicTransforms(is_train=True)
    
    def get_validation_transforms():
        return BasicTransforms(is_train=False)

# Test dataset creation
try:
    train_dataset = OilSpillDataset(data_root / 'train' / 'images',
                               data_root / 'train' / 'masks',
                               transform=get_training_transforms())
    
    val_dataset = OilSpillDataset(data_root / 'val' / 'images',
                             data_root / 'val' / 'masks',
                             transform=get_validation_transforms())
    
    print(f"✅ Train dataset: {len(train_dataset)} samples")
    print(f"✅ Validation dataset: {len(val_dataset)} samples")
    
    # Test loading a sample
    image, mask = train_dataset[0]
    print(f"🖼️ Sample image shape: {image.shape}")
    print(f"🏷️ Sample mask shape: {mask.shape}")
    print(f"📊 Image value range: [{image.min():.2f}, {image.max():.2f}]")
    print(f"📊 Mask value range: [{mask.min():.2f}, {mask.max():.2f}]")
    
except Exception as e:
    print(f"⚠️ Dataset creation failed: {e}")

⚠️ Albumentations not available, using basic transformations
✅ Train dataset: 20 samples
✅ Validation dataset: 8 samples
🖼️ Sample image shape: torch.Size([3, 256, 256])
🏷️ Sample mask shape: torch.Size([1, 256, 256])
📊 Image value range: [0.04, 0.55]
📊 Mask value range: [0.00, 0.00]


### Task 3.3: U-Net Architecture Implementation

Implement U-Net architecture for satellite image segmentation:

In [3]:
class ConvBlock(nn.Module):
    """Double Convolution Block for U-Net"""
    
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        return self.conv(x)

class UNet(nn.Module):
    """U-Net architecture for oil spill segmentation
    
    Paper: "U-Net: Convolutional Networks for Biomedical Image Segmentation"
    Link: https://arxiv.org/abs/1505.04597
    
    Modified for oil spill detection with variable input channels.
    """
    
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        """Initialize U-Net model.
        
        Args:
            in_channels: Number of input channels (3 for RGB, 1 for SAR)
            out_channels: Number of output channels (1 for binary segmentation)
            features: List of feature dimensions for each level
        """
        super(UNet, self).__init__()
        
        self.downs = nn.ModuleList()  # Downsampling (encoder) path
        self.ups = nn.ModuleList()    # Upsampling (decoder) path
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Encoder / Downsampling path
        for feature in features:
            self.downs.append(ConvBlock(in_channels, feature))
            in_channels = feature
        
        # Bottleneck
        self.bottleneck = ConvBlock(features[-1], features[-1] * 2)
        
        # Decoder / Upsampling path
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2)
            )
            self.ups.append(ConvBlock(feature * 2, feature))
        
        # Final convolution
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)
        
    def forward(self, x):
        """Forward pass through U-Net"""
        skip_connections = []
        
        # Encoder path - save skip connections
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        # Bottleneck
        x = self.bottleneck(x)
        
        # Reverse skip connections list for decoder path
        skip_connections = skip_connections[::-1]
        
        # Decoder path
        for idx in range(0, len(self.ups), 2):
            # Upsampling
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]
            
            # Handle cases where dimensions don't match
            if x.shape != skip_connection.shape:
                x = nn.functional.interpolate(
                    x, size=skip_connection.shape[2:], mode="bilinear", align_corners=True
                )
            
            # Concatenate with skip connection
            concat_skip = torch.cat((skip_connection, x), dim=1)
            
            # Convolution after concatenation
            x = self.ups[idx + 1](concat_skip)
        
        # Final 1x1 convolution for segmentation
        return self.final_conv(x)

# Test model creation
def test_unet(batch_size=2, in_channels=3, out_channels=1):
    """Test U-Net architecture"""
    x = torch.randn((batch_size, in_channels, 256, 256))
    model = UNet(in_channels=in_channels, out_channels=out_channels)
    preds = model(x)
    print(f"Input shape: {x.shape}")
    print(f"Output shape: {preds.shape}")
    
    # Calculate model size
    model_params = sum(p.numel() for p in model.parameters())
    print(f"Model parameters: {model_params:,}")
    
    # Move model to device
    model = model.to(device)
    print(f"Model moved to {device}")
    
    return model

# Create and test U-Net model
model = test_unet(batch_size=2, in_channels=3, out_channels=1)

Input shape: torch.Size([2, 3, 256, 256])
Output shape: torch.Size([2, 1, 256, 256])
Model parameters: 31,043,521
Model moved to cpu


### Task 3.4: Loss Functions and Evaluation Metrics

In [4]:
# Implement specialized loss functions for segmentation
class DiceLoss(nn.Module):
    """Dice Loss for segmentation"""
    
    def __init__(self, smooth=1e-5):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        # Flatten predictions and targets
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        
        # Apply sigmoid to predictions for binary segmentation
        predictions = torch.sigmoid(predictions)
        
        # Calculate Dice coefficient
        intersection = (predictions * targets).sum()
        dice = (2.0 * intersection + self.smooth) / (
            predictions.sum() + targets.sum() + self.smooth
        )
        
        return 1 - dice

# Combined BCE and Dice Loss for better segmentation results
class BCEDiceLoss(nn.Module):
    """Combined Binary Cross Entropy and Dice Loss"""
    
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(BCEDiceLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.dice_loss = DiceLoss()
    
    def forward(self, predictions, targets):
        bce_loss = self.bce_loss(predictions, targets)
        dice_loss = self.dice_loss(predictions, targets)
        combined_loss = self.bce_weight * bce_loss + self.dice_weight * dice_loss
        return combined_loss

# Evaluation metrics
def calculate_iou(pred_mask, gt_mask, threshold=0.5, smooth=1e-5):
    """Calculate IoU (Intersection over Union) metric
    
    Args:
        pred_mask: Predicted mask (after sigmoid)
        gt_mask: Ground truth mask
        threshold: Threshold for binary prediction
        smooth: Smoothing factor to avoid division by zero
    """
    # Apply threshold to get binary prediction
    pred_binary = (pred_mask > threshold).float()
    
    # Calculate intersection and union
    intersection = (pred_binary * gt_mask).sum()
    union = pred_binary.sum() + gt_mask.sum() - intersection
    
    # Calculate IoU
    iou = (intersection + smooth) / (union + smooth)
    
    return iou.item()

def calculate_dice(pred_mask, gt_mask, threshold=0.5, smooth=1e-5):
    """Calculate Dice coefficient metric"""
    # Apply threshold to get binary prediction
    pred_binary = (pred_mask > threshold).float()
    
    # Calculate intersection
    intersection = (pred_binary * gt_mask).sum()
    
    # Calculate Dice coefficient
    dice = (2.0 * intersection + smooth) / (
        pred_binary.sum() + gt_mask.sum() + smooth
    )
    
    return dice.item()

def calculate_metrics(pred_masks, gt_masks, threshold=0.5):
    """Calculate all evaluation metrics"""
    # Apply sigmoid for predictions if they're raw logits
    pred_masks = torch.sigmoid(pred_masks)
    
    batch_size = pred_masks.size(0)
    ious = []
    dices = []
    
    for i in range(batch_size):
        iou = calculate_iou(pred_masks[i], gt_masks[i], threshold)
        dice = calculate_dice(pred_masks[i], gt_masks[i], threshold)
        
        ious.append(iou)
        dices.append(dice)
    
    # Average metrics over batch
    avg_iou = sum(ious) / len(ious)
    avg_dice = sum(dices) / len(dices)
    
    return {
        'iou': avg_iou,
        'dice': avg_dice
    }

# Test loss functions
def test_loss_functions():
    """Test loss functions and metrics"""
    # Create dummy predictions and targets
    batch_size = 3
    h, w = 32, 32
    predictions = torch.randn(batch_size, 1, h, w)  # Raw logits
    targets = torch.randint(0, 2, (batch_size, 1, h, w)).float()  # Binary masks
    
    # Apply sigmoid to get probabilities
    pred_probs = torch.sigmoid(predictions)
    
    # Calculate BCE loss
    bce_loss = nn.BCEWithLogitsLoss()(predictions, targets)
    print(f"BCE Loss: {bce_loss.item():.4f}")
    
    # Calculate Dice loss
    dice_loss = DiceLoss()(predictions, targets)
    print(f"Dice Loss: {dice_loss.item():.4f}")
    
    # Calculate combined BCE-Dice loss
    bce_dice_loss = BCEDiceLoss()(predictions, targets)
    print(f"BCE-Dice Loss: {bce_dice_loss.item():.4f}")
    
    # Calculate evaluation metrics
    metrics = calculate_metrics(predictions, targets)
    print(f"IoU: {metrics['iou']:.4f}")
    print(f"Dice: {metrics['dice']:.4f}")

# Run test
test_loss_functions()

BCE Loss: 0.8012
Dice Loss: 0.4979
BCE-Dice Loss: 0.6496
IoU: 0.3404
Dice: 0.5078


## Module 4: Training and Evaluation

### Task 4.1: Training Function Implementation

In [ ]:
# Create DataLoaders for training and validation
def create_dataloaders(batch_size=8):
    """Create DataLoaders for training and validation"""
    train_dataset = OilSpillDataset(data_root / 'train' / 'images',
                                  data_root / 'train' / 'masks',
                                  transform=get_training_transforms())
    
    val_dataset = OilSpillDataset(data_root / 'val' / 'images',
                                data_root / 'val' / 'masks',
                                transform=get_validation_transforms())
    
    # Set num_workers=0 to avoid multiprocessing issues on Windows
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=0, pin_memory=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                            num_workers=0, pin_memory=False)
    
    return train_loader, val_loader

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, 
                num_epochs=10, device='cpu'):
    """Train the model
    
    Args:
        model: PyTorch model
        train_loader: Training data loader
        val_loader: Validation data loader
        criterion: Loss function
        optimizer: Optimizer
        scheduler: Learning rate scheduler (optional)
        num_epochs: Number of epochs to train
+        device: Device to train on ('cuda' or 'cpu')
    """
    # Initialize tracking variables
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_iou': [],
        'val_dice': []
    }
    
    best_dice = 0.0
    
    # Loop over epochs
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 15)
        
        # Training phase
        model.train()
        train_loss = 0.0
        
        for batch_idx, (images, masks) in enumerate(train_loader):
            # Move to device
            images = images.to(device)
            masks = masks.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Update loss
            train_loss += loss.item() * images.size(0)
            
            # Print progress
            if (batch_idx + 1) % 5 == 0 or (batch_idx + 1) == len(train_loader):
                print(f"Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
        
        # Calculate average training loss
        train_loss = train_loss / len(train_loader.dataset)
        history['train_loss'].append(train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_ious = []
        val_dices = []
        
        with torch.no_grad():
            for images, masks in val_loader:
                # Move to device
                images = images.to(device)
                masks = masks.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                # Update loss
                val_loss += loss.item() * images.size(0)
                
                # Calculate metrics
                metrics = calculate_metrics(outputs, masks)
                val_ious.append(metrics['iou'])
                val_dices.append(metrics['dice'])
        
        # Calculate average validation metrics
        val_loss = val_loss / len(val_loader.dataset)
        val_iou = sum(val_ious) / len(val_ious) if val_ious else 0.0
        val_dice = sum(val_dices) / len(val_dices) if val_dices else 0.0
        
        history['val_loss'].append(val_loss)
        history['val_iou'].append(val_iou)
        history['val_dice'].append(val_dice)
        
        # Update learning rate scheduler if provided
        if scheduler is not None:
            scheduler.step(val_loss)
        
        # Print epoch results
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Val IoU: {val_iou:.4f}, "
              f"Val Dice: {val_dice:.4f}")
        
        # Save best model
        if val_dice > best_dice:
            best_dice = val_dice
            print(f"New best model with Dice: {best_dice:.4f}!")
            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_dice': best_dice,
                'val_iou': val_iou,
            }, '../models/best_model.pth')
        
        print()
    
    return model, history

### Task 4.2: Train the Model

In [ ]:
# Create directory for model checkpoints
Path('../models').mkdir(exist_ok=True)

# Hyperparameters
BATCH_SIZE = 8
LEARNING_RATE = 1e-3
NUM_EPOCHS = 5  # Reduced for demo
IN_CHANNELS = 3  # RGB images
OUT_CHANNELS = 1  # Binary segmentation

# Create dataloaders
train_loader, val_loader = create_dataloaders(batch_size=BATCH_SIZE)
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

# Create model
model = UNet(in_channels=IN_CHANNELS, out_channels=OUT_CHANNELS)
model = model.to(device)

# Loss function and optimizer
criterion = BCEDiceLoss(bce_weight=0.5, dice_weight=0.5)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler (fixed verbose parameter)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=3
)

# Train the model
print("🚀 Starting model training...")
model, history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=NUM_EPOCHS,
    device=device
)
print("✅ Training complete!")

Training batches: 3
Validation batches: 1
🚀 Starting model training...
Epoch 1/5
---------------
🚀 Starting model training...
Epoch 1/5
---------------
Batch 3/3, Loss: 0.6144
Batch 3/3, Loss: 0.6144
Epoch 1/5 - Train Loss: 0.7091, Val Loss: 0.8094, Val IoU: 0.0634, Val Dice: 0.1123
New best model with Dice: 0.1123!
Epoch 1/5 - Train Loss: 0.7091, Val Loss: 0.8094, Val IoU: 0.0634, Val Dice: 0.1123
New best model with Dice: 0.1123!

Epoch 2/5
---------------

Epoch 2/5
---------------
Batch 3/3, Loss: 0.5445
Batch 3/3, Loss: 0.5445
Epoch 2/5 - Train Loss: 0.5574, Val Loss: 2.3000, Val IoU: 0.0634, Val Dice: 0.1124
New best model with Dice: 0.1124!
Epoch 2/5 - Train Loss: 0.5574, Val Loss: 2.3000, Val IoU: 0.0634, Val Dice: 0.1124
New best model with Dice: 0.1124!

Epoch 3/5
---------------

Epoch 3/5
---------------
Batch 3/3, Loss: 0.5196
Batch 3/3, Loss: 0.5196
Epoch 3/5 - Train Loss: 0.5200, Val Loss: 0.7772, Val IoU: 0.3048, Val Dice: 0.3410
New best model with Dice: 0.3410!
Epoch 

### Task 4.3: Plot Training Results

In [ ]:
def plot_training_history(history):
    """Plot training history with loss and metrics"""
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot loss curves
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss plot
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # IoU plot
    axes[1].plot(epochs, history['val_iou'], 'g-', label='Validation IoU')
    axes[1].set_title('Validation IoU')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('IoU')
    axes[1].legend()
    axes[1].grid(True)
    
    # Dice plot
    axes[2].plot(epochs, history['val_dice'], 'c-', label='Validation Dice')
    axes[2].set_title('Validation Dice Coefficient')
    axes[2].set_xlabel('Epochs')
    axes[2].set_ylabel('Dice Coefficient')
    axes[2].legend()
    axes[2].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print(f"Best validation IoU: {max(history['val_iou']):.4f}")
    print(f"Best validation Dice: {max(history['val_dice']):.4f}")

# Plot training history
plot_training_history(history)

### Task 4.4: Model Evaluation and Prediction Visualization

In [ ]:
def load_best_model():
    """Load the best model checkpoint"""
    # Create a new model instance
    model = UNet(in_channels=IN_CHANNELS, out_channels=OUT_CHANNELS).to(device)
    
    # Load checkpoint
    checkpoint = torch.load('../models/best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Loaded best model from epoch {checkpoint['epoch']+1} ")
    print(f"Validation Dice: {checkpoint['val_dice']:.4f}, Validation IoU: {checkpoint['val_iou']:.4f}")
    
    return model

def visualize_predictions(model, dataloader, num_samples=5):
    """Visualize model predictions"""
    model.eval()
    
    # Get samples from dataloader
    samples = []
    for images, masks in dataloader:
        batch_size = images.size(0)
        for i in range(batch_size):
            samples.append((images[i], masks[i]))
        if len(samples) >= num_samples:
            break
    
    # Select random samples
    samples = random.sample(samples, min(num_samples, len(samples)))
    
    # Visualize predictions
    fig, axes = plt.subplots(num_samples, 4, figsize=(20, num_samples * 5))
    fig.suptitle('Oil Spill Detection Results', fontsize=16)
    
    # Column titles
    titles = ['Input Image', 'Ground Truth Mask', 'Predicted Mask', 'Overlay']
    for j, title in enumerate(titles):
        axes[0, j].set_title(title, fontsize=14)
    
    with torch.no_grad():
        for i, (image, mask) in enumerate(samples):
            # Make prediction
            image_tensor = image.unsqueeze(0).to(device)  # Add batch dimension
            output = model(image_tensor)
            pred_mask = torch.sigmoid(output).squeeze().cpu().numpy()
            pred_binary = (pred_mask > 0.5).astype(np.float32)
            
            # Convert tensors to numpy for visualization
            if has_albumentation:
                # Denormalize image
                image_np = image.cpu().numpy().transpose(1, 2, 0)  # (C,H,W) -> (H,W,C)
                std = np.array([0.229, 0.224, 0.225])
                mean = np.array([0.485, 0.456, 0.406])
                image_np = image_np * std + mean
                image_np = np.clip(image_np, 0, 1)
            else:
                image_np = image.cpu().numpy().transpose(1, 2, 0)  # (C,H,W) -> (H,W,C)
            
            mask_np = mask.squeeze().cpu().numpy()  # (1,H,W) -> (H,W)
            
            # Create overlay
            overlay = image_np.copy()
            overlay[pred_binary > 0.5] = [1, 0, 0]  # Red for predicted oil spill
            
            # Plot results
            axes[i, 0].imshow(image_np)
            axes[i, 0].set_xticks([])
            axes[i, 0].set_yticks([])
            
            axes[i, 1].imshow(mask_np, cmap='gray')
            axes[i, 1].set_xticks([])
            axes[i, 1].set_yticks([])
            
            axes[i, 2].imshow(pred_binary, cmap='gray')
            axes[i, 2].set_xticks([])
            axes[i, 2].set_yticks([])
            
            axes[i, 3].imshow(overlay)
            axes[i, 3].set_xticks([])
            axes[i, 3].set_yticks([])
            
            # Calculate and display metrics
            iou = calculate_iou(torch.tensor(pred_binary), torch.tensor(mask_np))
            dice = calculate_dice(torch.tensor(pred_binary), torch.tensor(mask_np))
            axes[i, 3].set_xlabel(f"IoU: {iou:.4f}, Dice: {dice:.4f}")
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.9)
    plt.show()

# Load best model and evaluate
try:
    best_model = load_best_model()
    
    # Create test dataset
    test_dataset = OilSpillDataset(data_root / 'test' / 'images',
                                  data_root / 'test' / 'masks',
                                  transform=get_validation_transforms())
    
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    # Visualize predictions
    visualize_predictions(best_model, test_loader, num_samples=5)
    
except FileNotFoundError:
    print("⚠️ Best model checkpoint not found. Please train the model first.")
except Exception as e:
    print(f"⚠️ Error loading model: {e}")

### Task 4.5: Model Fine-tuning and Hyperparameter Optimization

After initial training, we can experiment with hyperparameter tuning to improve model performance.

In [ ]:
# Hyperparameter tuning experiments
def run_hyperparameter_experiment(experiment_name, params):
    """Run a hyperparameter experiment"""
    print(f"🧪 Running experiment: {experiment_name}")
    print(f"Parameters: {params}")
    
    # Create model with specified parameters
    model = UNet(in_channels=params['in_channels'], 
                out_channels=params['out_channels'],
                features=params.get('features', [64, 128, 256, 512]))
    model = model.to(device)
    
    # Loss function
    criterion = BCEDiceLoss(
        bce_weight=params.get('bce_weight', 0.5),
        dice_weight=params.get('dice_weight', 0.5)
    )
    
    # Optimizer
    if params['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    elif params['optimizer'] == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=params['learning_rate'], momentum=0.9)
    else:
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=3, verbose=True
    )
    
    # Create dataloaders
    train_loader, val_loader = create_dataloaders(batch_size=params['batch_size'])
    
    # Train the model
    model, history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=params['num_epochs'],
        device=device
    )
    
    # Save experiment results
    experiment_dir = Path(f"../experiments/{experiment_name}")
    experiment_dir.mkdir(parents=True, exist_ok=True)
    
    # Save model
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'params': params,
        'history': history,
        'best_dice': max(history['val_dice']),
        'best_iou': max(history['val_iou']),
    }, experiment_dir / 'model.pth')
    
    # Plot training history
    plot_training_history(history)
    
    return model, history, max(history['val_dice']), max(history['val_iou'])

# Define experiments to run
# Uncomment to run hyperparameter tuning
"""
experiments = [
    {
        'name': 'baseline',
        'params': {
            'in_channels': 3,
            'out_channels': 1,
            'batch_size': 8,
            'learning_rate': 1e-3,
            'optimizer': 'adam',
            'bce_weight': 0.5,
            'dice_weight': 0.5,
            'num_epochs': 15,
        }
    },
    {
        'name': 'smaller_lr',
        'params': {
            'in_channels': 3,
            'out_channels': 1,
            'batch_size': 8,
            'learning_rate': 1e-4,
            'optimizer': 'adam',
            'bce_weight': 0.5,
            'dice_weight': 0.5,
            'num_epochs': 15,
        }
    },
    {
        'name': 'more_dice_weight',
        'params': {
            'in_channels': 3,
            'out_channels': 1,
            'batch_size': 8,
            'learning_rate': 1e-3,
            'optimizer': 'adam',
            'bce_weight': 0.3,
            'dice_weight': 0.7,
            'num_epochs': 15,
        }
    },
]

# Create experiments directory
Path('../experiments').mkdir(exist_ok=True)

# Run experiments
results = {}
for experiment in experiments:
    model, history, best_dice, best_iou = run_hyperparameter_experiment(
        experiment['name'], experiment['params']
    )
    results[experiment['name']] = {
        'best_dice': best_dice,
        'best_iou': best_iou,
        'params': experiment['params']
    }

# Print experiment results
print("\n📊 Experiment Results:")
for name, result in results.items():
    print(f"Experiment: {name}")
    print(f"Best Dice: {result['best_dice']:.4f}, Best IoU: {result['best_iou']:.4f}")
    print(f"Learning Rate: {result['params']['learning_rate']}")
    print(f"BCE Weight: {result['params']['bce_weight']}, Dice Weight: {result['params']['dice_weight']}")
    print("-" * 50)

# Find best experiment
best_experiment = max(results.items(), key=lambda x: x[1]['best_dice'])
print(f"✨ Best experiment: {best_experiment[0]} with Dice: {best_experiment[1]['best_dice']:.4f}")
"""

print("\n✅ Module 3 and Module 4 complete! You have successfully implemented:")
print("✓ U-Net architecture for oil spill segmentation")
print("✓ Data loading pipeline with augmentation")
print("✓ Loss functions (BCE-Dice) and evaluation metrics (IoU, Dice)")
print("✓ Model training and evaluation pipeline")
print("✓ Hyperparameter tuning framework")

## Summary and Next Steps

In this notebook, we successfully implemented Weeks 3-4 requirements for the AI SpillGuard Oil Spill Detection project:

1. **Module 3: Model Development (Segmentation and Classification)**
   - Designed and implemented U-Net architecture for oil spill segmentation
   - Customized input layers to handle satellite imagery
   - Built complete segmentation pipeline

2. **Module 4: Training and Evaluation**
   - Implemented training with real-time augmentation
   - Created custom loss functions (BCE, Dice, BCE-Dice)
   - Added comprehensive evaluation metrics (IoU, Dice)
   - Created framework for hyperparameter optimization

**Next steps:**
- Deploy model for real-time oil spill detection
- Implement additional CNN architectures for comparison (DeepLabV3+, SegNet)
- Add post-processing techniques to improve segmentation accuracy
- Optimize model for edge deployment on marine vessels or satellites